In [ ]:
import numpy as np
import ioh

In [ ]:
ioh.ProblemClass.GRAPH.problems

# First idea 
as dealing with submodular functions, we may want to select the arguments that make it grow "the most at first" --> as when adding more elements we face diminishing returns
so from a solution x -> we build lam xs turning off L bits of uar and keeping the best -> we then build lam ys by flipping on L' bits of the best xs uniformly at random and keeping the best y --> then we turn on each bit present in y and not in x

problem: solution x keeps growing?
how to stop that?

In [ ]:
import random
class OffAndOn:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, init_a_0 : bool, c : float, pc : float):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.c: bool = c
        self.pc: bool = pc
        self.init_a_0: bool = init_a_0
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables

        if self.init_a_0:
            x = np.zeros(dim, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, dim)
            val = problem(x)
        
        for _ in range(self.n):
            L = np.random.binomial(dim, self.p_off)
            
            xp = None
            best_val = None 
            # for _ in range(self.lam):
            for _ in range(1):
                # mutation 1 : we turn off L bits 
                bits_to_flip = np.array(random.sample(range(dim), L))
                new_xp = np.array(x.copy())
                new_xp[bits_to_flip] = 0
                val_new_xp = problem(new_xp)
                if best_val is None or val_new_xp > best_val:
                    xp = new_xp
                    best_val = val_new_xp 

            L = np.random.binomial(dim, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = xp.copy()
                bits_to_flip = np.array(random.sample(range(dim), L))
                new_y[bits_to_flip] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            
            mix = [(1 if y[i] else (y[i] if np.random.binomial(1, self.c) else x[i])) 
                   for i in range(dim)]
            val_mix = problem(mix)
            if val_mix > val:
                x = mix
                val = val_mix
                # self.p_on = max(10/dim, self.p_on / self.pc)
                self.p_off = max(10/dim, self.p_off / self.pc)
                # self.c = max(10/dim, self.c / self.pc)
            

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn(1000, lam, 1/4, 10/dim_pb, False, 1/2, 2)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 8
offandon = OffAndOn(1200, lam, 1/8, 10/dim_pb, True, 1/4, 1.05)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Second idea:

from x:
 - generate lam children by flipping bits randomly 
 - take the lam2 best and do the intersection --> those are the "best elements to keep"
 - combine x and the intersection

In [ ]:
class IntersectAndCombine:
    def __init__(self, n: int, lam: int, lam2: int, p: float, c: float, pc: float, init_a_0 : bool):
        self.n: int = n
        self.p: int = p
        self.c: int = c
        self.lam: float = lam
        self.lam2: float = lam2
        self.init_a_0: float = init_a_0
        self.pc: float = pc

    def generate_child(self, x):
        y = x.copy()
        for ind in range(len(x)):
            y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
        return y

    def intersection(self, x, y):
        return np.logical_and(x, y).astype(int)

    def union(self, x, y):
        return np.logical_or(x, y).astype(int)

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.init_a_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, problem.meta_data.n_variables)
            val = problem(x)
        for tour in range(self.n):
            values = []
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x)
                values.append(problem(x_))
                children.append(x_)

            children = np.array(children)
            sorted_inds = np.argsort(values)[-self.lam2:]
            children = children[sorted_inds]
            intersection = np.logical_and.reduce(children).astype(int) # intersection will be small

            mix = [(1 if intersection[i] else (intersection[i] if np.random.binomial(1, self.c) else x[i])) 
                   for i in range(problem.meta_data.n_variables)]
            val_mix = problem(mix)

            union = self.union(intersection.copy(),x.copy())
            val_u = problem(union)

            elems = [x, mix, union]
            vals = [val, val_mix, val_u]

            ind = np.argmax(vals)
            
            val = vals[ind]
            x = elems[ind]  
            if (ind != 0):
                self.p = max(10/problem.meta_data.n_variables, self.p / self.pc)
                self.c = max(10/problem.meta_data.n_variables, self.c / self.pc)
                # print(tour, "update", self.p, "  ind ", ind) 

In [ ]:
import math
max_cut = ioh.get_problem(2003, problem_class=ioh.ProblemClass.GRAPH)
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 8
intcomb = IntersectAndCombine(1000, lam, 3, 1/4, 1/4, 1.005, False)
intcomb(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 8
intcomb = IntersectAndCombine(1000, lam, 5, 1/4, 10/dim_pb, 1.005, True)
intcomb(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Third idea (QB):

from x:
 - generate lam children by flipping bits randomly 
 - take the lam2 best and build a prob distr proportionnally to their obj values (TODO: see how to integrate sol x)
 - "crossover"?

In [ ]:
class ProbDistr:
    def __init__(self, n: int, lam: int, p: float, c: float, pc: float, init_a_0 : bool):
        self.n: int = n
        self.p: int = p
        self.c: int = c
        self.pc: int = pc
        self.lam: float = lam
        self.init_a_0: float = init_a_0

    def generate_child(self, x):
        y = x.copy()
        for ind in range(len(x)):
            y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
        return y

    def get_new_from_children(self, children, values, dim):
        values = np.array(list(map(lambda x : max(x,0), values)))
        if (not values.all()): # means all values are negative
            values = np.array([1/len(children)]*(len(children)))
        sum = np.sum(values)
        p = [v/sum for v in values]
        new_x = np.zeros(dim).astype(int)
        for ind in range(dim):
            chosen_child = np.random.choice(np.arange(0, len(children)), p=p) 
            new_x[ind] = children[chosen_child][ind]
        return new_x

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.init_a_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, problem.meta_data.n_variables)
            val = problem(x)
        for tour in range(self.n):
            values = []
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x)
                values.append(problem(x_))
                children.append(x_)

            new_x = self.get_new_from_children(np.array(children), np.array(values), problem.meta_data.n_variables)
            val_new_x = problem(new_x)

            # children.append(x)
            # values.append(val)
            # crossed_x_new_way = self.get_new_from_children(np.array(children), np.array(values), problem.meta_data.n_variables)
            # val_crossed_x_new_way = problem(crossed_x_new_way)

            mix = [new_x[i] if np.random.binomial(1, self.c) else x[i] for i in range(problem.meta_data.n_variables)]
            val_mix = problem(mix)

            # ind = np.argmax([val, val_mix, val_new_x, val_crossed_x_new_way])
            ind = np.argmax([val, val_mix, val_new_x])
            
            # val = [val, val_mix, val_new_x, val_crossed_x_new_way][ind]
            val = [val, val_mix, val_new_x][ind]
            # x = [x, mix, new_x, crossed_x_new_way][ind]  
            x = [x, mix, new_x][ind]  
            if (ind != 0):
                self.p = max(10/problem.meta_data.n_variables, self.p / self.pc)
                self.c = max(10/problem.meta_data.n_variables, self.c / self.pc)
                print(tour, "update", self.p, "  ind ", ind) 

In [ ]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
pd = ProbDistr(1000, lam, 1/4, 10/dim_pb, 1.005, False)
pd(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 8
pd = ProbDistr(1200, lam, 1/4, 10/dim_pb, True)
pd(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Algo par croissance

In [ ]:
class ParCroissance:

    def __init__(self, n: int, s: int, k: int, init_a_0: bool, pc : float):
        self.n: int = n  # nb d'itérations
        self.s: int = s  # population size
        self.k: int = k  # taux de croissance
        self.pc: int = pc  # taux de croissance
        self.init_a_0: int = init_a_0  # taux de croissance

    def flip(self, x, l):
        positions = np.random.choice(np.arange(0, len(x)), l, replace=False)
        y = x.copy()
        y[positions] = 1 - y[positions]
        return y

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables

        # Initialization
        if self.init_a_0:
            x = np.zeros(dim, dtype=int)
            y = problem(x)
        else:
            x = np.random.randint(0, 2, dim)
            y = problem(x)

        # Optimization
        for t in range(self.n-1):
            # On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = self.flip(x, self.k)
                yy = problem(xx)
                pop.append((xx, yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
            else:
                self.k = max(int(self.k * self.pc), 1)
                if (self.k == 1):
                    print(t,"1")

In [ ]:
import math
pb = ioh.get_problem(2001, problem_class=ioh.ProblemClass.GRAPH)
pb.reset()
dim_pb = pb.meta_data.n_variables
pc = ParCroissance(1500, 8, 30, False, 3/4)
pc(pb)
print(pb.state.evaluations, pb.state.y_unconstrained_best)

In [ ]:
import math
max_coverage.reset()
dim_pb = max_cut.meta_data.n_variables
pc = ParCroissance(1000, 10, 30, True, 5/6)
pc(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

# Fast Greedy

In [161]:
import queue


class FastGreedy:

    def __init__(self, budget: int, num_1_start: int):
        self.num_1_start = num_1_start
        self.budget = budget

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        x = np.zeros(dim).astype(int)
        if (self.num_1_start > 0):
            x[np.array(random.sample(range(dim), self.num_1_start))] = 1
        val = problem(x)

        evals = 0

        pq = queue.PriorityQueue()  # elems have type (marginal gain of elem index,index)
        inds = np.where(x == 0)[0]
        np.random.shuffle(inds)
        for e in inds:
            x[e] = 1
            # - to retrieve elems with highest marginal gain
            pq.put((-problem(x)+val, e))
            evals += 1
            x[e] = 0

        x_best = x
        val_best = val

        while (evals <= self.budget and not pq.empty()):
            first_e = pq.get()
            # print(first_e)
            x[first_e[1]] = 1
            new_val = -problem(x)+val
            evals += 1
            x[first_e[1]] = 0
            pq.put((new_val, first_e[1]))
            current_best = first_e[1]
            elem = pq.get()
            # if elem[0] > new_val then no point in keeping the search going
            while (elem[0] < new_val and evals <= self.budget and not pq.empty()):
                x[elem[1]] = 1
                aux = -problem(x)+val
                evals += 1
                x[elem[1]] = 0
                pq.put((aux, elem[1]))
                if aux < new_val:
                    new_val = aux
                    current_best = elem[1]
                elem = pq.get()
            x[current_best] = 1
            val = problem(x)
            # print(val)
        return x

In [ ]:
import math
pb = ioh.get_problem(2002, problem_class=ioh.ProblemClass.GRAPH)
pb.reset()
dim_pb = pb.meta_data.n_variables
pc = FastGreedy(10000, 0)
pc(pb)
print(pb.state.evaluations, pb.state.y_unconstrained_best)

In [ ]:
import math
pb = ioh.get_problem(2103, problem_class=ioh.ProblemClass.GRAPH)
pb.reset()
dim_pb = pb.meta_data.n_variables
pc = FastGreedy(10000, 0)
pc(pb)
print(pb.state.evaluations, pb.state.y_unconstrained_best)

# Utils

In [151]:
def run_experiment(problem, algorithm, n_runs=5,s="problem:"):
    for run in range(n_runs):
        algorithm(problem)
        print(s,problem.state.evaluations, problem.state.y_unconstrained_best)
        problem.reset()

In [ ]:
import os

logger_on_off = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="test_data",       # in a folder named: 'my-experiment'
    algorithm_name="on_off",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

ioh.ProblemClass.GRAPH.problems
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_on_off)
    dim_pb = problem.meta_data.n_variables
    lam = 10
    offandon = OffAndOn(math.ceil(10000 / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True, lam/dim_pb)
    run_experiment(problem, offandon, n_runs=1)

logger_on_off.close()

In [ ]:
logger_int_comb = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="test_data_int_comb",       
    algorithm_name="int_comb",    
    store_positions=True               
)

logger_int_comb

print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)
for i in range(2100,2128):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_int_comb)
    dim_pb = max_coverage.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, intcomb, n_runs=1)

logger_int_comb.close()

In [ ]:
logger_int_comb_max_cut = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="test_data_int_comb_max_cut",       
    algorithm_name="int_comb",    
    store_positions=True               
)

logger_int_comb_max_cut

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_int_comb_max_cut)
    dim_pb = max_coverage.meta_data.n_variables
    lam = 20
    intcomb = IntersectAndCombine(500, lam, 5, 1/4, 10/dim_pb, True)
    run_experiment(problem, offandon, n_runs=1)

logger_int_comb_max_cut.close()

In [ ]:
# 2000-2005 max_cut
max_cut = range(2000,20005)
# 2300-2304 packwhiletravel
# 2300-2304 packwhiletravel
for i in max_cut:
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    dim_pb = problem.meta_data.n_variables
    lam = 8
    intcomb = IntersectAndCombine(1000, lam, 3, 1/4, 10/dim_pb, 1.01, True)
    run_experiment(problem, intcomb, n_runs=1,s="intcomb:")
    problem.reset()
    # ea = EA_1p1(nb_it, 1/dim_pb)
    # run_experiment(problem, ea, n_runs=1,s="ea:")
    # problem.reset()

# Yes Distant

In [ ]:
import random
class Distance:
    def __init__(self, n: int, k : int, init_a_0 : bool, c : float):
        self.n: int = n
        self.k: int = k
        # self.p_on: int = p_on
        # self.lam: float = lam
        self.c: bool = c
        # self.pc: bool = pc
        self.init_a_0: bool = init_a_0
        
    def get_num_1_at_dist_k(self, x, ind):
        return np.count_nonzero(x[max(ind-self.k,0):min(ind+self.k,self.dim)])

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        self.dim = problem.meta_data.n_variables

        if self.init_a_0:
            x = np.zeros(self.dim, dtype=int)
            val = problem(x)
        else:
            x = np.random.randint(0, 2, self.dim)
            val = problem(x)
        
        for it in range(self.n):
            weights = [self.get_num_1_at_dist_k(x, ind) for ind in range(self.dim)]
            sum_weights = np.sum(weights) 
            child = np.zeros(self.dim).astype(int)
            for ind in range(self.dim):
                prob = weights[ind]/sum_weights if sum_weights != 0 else 1/self.dim
                child[ind] = x[ind] if np.random.binomial(1, prob) else 1 - x[ind]
            v_child = problem(child)
            mix = [child[i] if np.random.binomial(1, self.c) else x[i] for i in range(self.dim)]
            val_mix = problem(mix)

            elems = [x, mix, child]
            vals = [val, val_mix, v_child]
            ind = np.argmax(vals)
            val = vals[ind]
            x = elems[ind]  
            if (ind != 0):
                pass
                # print(it, ind)
                # self.p = max(10/problem.meta_data.n_variables, self.p / self.pc)
                # self.c = max(10/problem.meta_data.n_variables, self.c / self.pc)            
             

In [ ]:
pb = ioh.get_problem(2002, problem_class=ioh.ProblemClass.GRAPH)
pb.reset()
dim_pb = pb.meta_data.n_variables
pc = Distance(5000, 10, False, 10/dim_pb)
pc(pb)
print(pb.state.evaluations, pb.state.y_unconstrained_best)

# Combining stuff

In [166]:
class DistAndCombine:
    def __init__(self, n: int, lam: int, lam2: int, init_a_0 : bool, k : int):
        self.n: int = n
        self.k: int = k
        self.lam: float = lam
        self.lam2: float = lam2
        self.init_a_0: float = init_a_0

    def get_num_1_at_dist_k(self, x, ind):
        return np.count_nonzero(x[max(ind-self.k,0):min(ind+self.k,self.dim)])
    
    def generate_child(self, x, weights, sum_weights):
        child = np.zeros(self.dim).astype(int)
        for ind in range(self.dim):
            prob = weights[ind]/sum_weights if sum_weights != 0 else 1/self.dim
            child[ind] = 1 - x[ind] if np.random.binomial(1, prob) else x[ind]
        return child
    
    def get_new_from_children(self, children, values, dim):
        values = np.array(list(map(lambda x : max(x,0), values)))
        if (not values.all()): # means all values are negative
            values = np.array([1/len(children)]*(len(children)))
        sum = np.sum(values)
        p = [v/sum for v in values]
        new_x = np.zeros(dim).astype(int)
        for ind in range(dim):
            chosen_child = np.random.choice(np.arange(0, len(children)), p=p) 
            new_x[ind] = children[chosen_child][ind]
        return new_x

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        modif = False
        self.dim = problem.meta_data.n_variables
        # if self.init_a_0:
        #     x = np.zeros(self.dim, dtype=int)
        #     val = problem(x)
        # else:
        #     x = np.random.randint(0, 2, self.dim)
        #     val = problem(x)
        x = FastGreedy(10000, 0)(problem)
        print(x)
        problem.reset()
        val = problem(x)
        print(val)

        for _ in range(self.n):    
            weights = [self.get_num_1_at_dist_k(x, ind) for ind in range(self.dim)]
            sum_weights = np.sum(weights) 
            values = []
            children = []
            for _ in range(self.lam):
                x_ = self.generate_child(x, weights, sum_weights)
                values.append(problem(x_))
                children.append(x_)

            children = np.array(children)
            sorted_inds = np.argsort(values)[-self.lam2:]
            children = children[sorted_inds]
            values = np.array(values)[sorted_inds]

            new_x = self.get_new_from_children(np.array(children), np.array(values), self.dim)
            val_new_x = problem(new_x)

            np.append(x,children)
            np.append(val,values)
            crossed_x_new_way = self.get_new_from_children(np.array(children), np.array(values), self.dim)
            val_crossed_x_new_way = problem(crossed_x_new_way)
            
            elems = [x, new_x, crossed_x_new_way]
            vals = [val, val_new_x, val_crossed_x_new_way]

            ind = np.argmax(vals)
            
            val = vals[ind]
            x = elems[ind] 

In [168]:
for i in range(2000,2005):
    pb = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    pb.reset()
    dim_pb = pb.meta_data.n_variables
    lam = 10
    intdist = DistAndCombine(500, lam, 2, False, 5)
    intdist(pb)
    print(pb.state.evaluations, pb.state.y_unconstrained_best)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

KeyboardInterrupt: 

In [160]:
import os

logger_dist_comb = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="distcomb_res",       # in a folder named: 'my-experiment'
    algorithm_name="distcomb",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2102,2104):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_dist_comb)
    dim_pb = problem.meta_data.n_variables
    intdist = DistAndCombine(2000, 8, 1, False, 5)
    run_experiment(problem, intdist, n_runs=1)

logger_dist_comb.close()

problem: 20001 546.0
problem: 20001 653.0
